Importa bibliotecas

In [40]:

import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
import sqlalchemy as sa


Coleta dados e prepara o dataframe do Arquivo de taxas

In [2]:
df_taxas = pd.read_csv("Dados/tabela_taxas.csv")

In [3]:
df_taxas['periodo']  = pd.to_datetime(df_taxas['periodo'])
df_taxas['ano_mes']  = df_taxas['periodo'].dt.strftime('%Y/%m')
df_taxas['mao_de_obra_indireta'] = pd.to_numeric(df_taxas['mao_de_obra_indireta'])
df_taxas['energia'] = pd.to_numeric(df_taxas['energia'])
df_taxas['manutencao'] = pd.to_numeric(df_taxas['manutencao'])
df_taxas['depreciacao'] = pd.to_numeric(df_taxas['depreciacao'])

C:\Users\Anderson\AppData\Local\Temp\ipykernel_15104\1309914708.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_taxas['periodo']  = pd.to_datetime(df_taxas['periodo'])


Coleta dados e prepara o dataframe do Arquivo de Custos

In [4]:
df_custos = pd.read_csv("Dados/tabela_custos.csv")

In [5]:
df_custos['periodo']  = pd.to_datetime(df_custos['periodo'])
df_custos['ano_mes']  = df_custos['periodo'].dt.strftime('%Y/%m')
df_custos['mao_de_obra'] = pd.to_numeric(df_custos['mao_de_obra'])
df_custos['materia_prima'] = pd.to_numeric(df_custos['materia_prima'])
df_custos['gasto_geral'] = pd.to_numeric(df_custos['gasto_geral'])

C:\Users\Anderson\AppData\Local\Temp\ipykernel_15104\1860566583.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_custos['periodo']  = pd.to_datetime(df_custos['periodo'])


Unir dataframes

In [6]:
df_custos.head()

,cod_item,periodo,mao_de_obra,materia_prima,gasto_geral,ano_mes
0,101,2021-01-01,864.54,163.20,175.16,2021/01
1,101,2021-02-01,309.56,633.97,405.12,2021/02
2,101,2021-03-01,496.58,1101.79,64.84,2021/03
3,101,2021-04-01,232.84,595.31,82.95,2021/04
4,101,2021-05-01,984.27,1000.81,459.82,2021/05


In [7]:
df_merge = pd.merge(df_custos,df_taxas, on='ano_mes')

In [8]:
df_merge = df_merge.drop(columns=["periodo_y"])
df_merge = df_merge.rename(columns={'periodo_x':'periodo','cod_item':'cod_produto'})

In [9]:
df_merge.head()

,cod_produto,periodo,mao_de_obra,materia_prima,gasto_geral,ano_mes,mao_de_obra_indireta,energia,manutencao,depreciacao
0,101,2021-01-01,864.54,163.20,175.16,2021/01,0.384744,0.297201,0.178445,0.139609
1,102,2021-01-01,830.05,650.47,377.59,2021/01,0.384744,0.297201,0.178445,0.139609
2,103,2021-01-01,548.86,1222.01,109.75,2021/01,0.384744,0.297201,0.178445,0.139609
3,104,2021-01-01,928.17,6.48,56.61,2021/01,0.384744,0.297201,0.178445,0.139609
4,105,2021-01-01,560.99,667.31,497.53,2021/01,0.384744,0.297201,0.178445,0.139609


Coleta dados e prepara o dataframe do Arquivo de Produtos

In [10]:
df_produtos = pd.read_excel("Dados/Produto.xlsx")

In [11]:
df_produtos['peso'] = pd.to_numeric(df_produtos['Peso (kg)'])
df_produtos['valor_unitario'] = pd.to_numeric(df_produtos['valor_unitario'])
df_produtos['cod_produto'] = pd.to_numeric(df_produtos['cod_Produto'])
df_produtos = df_produtos.drop(columns=["Produto","Categoria","Peso (kg)","cod_Produto"])

Coleta dados e prepara o dataframe do Arquivo de Vendas

In [12]:
dataframe = []
for file in os.listdir("Dados/Vendas"):
    if file.endswith(".csv"):
        file_path = os.path.join("Dados/Vendas", file)
        df_file = pd.read_csv(file_path)
        dataframe.append(df_file)
df_vendas = pd.concat(dataframe,ignore_index=True)

In [13]:
df_vendas['data_venda']  = pd.to_datetime(df_vendas['data_venda'],format='%d-%m-%Y')
df_vendas['ano_mes']  = df_vendas['data_venda'].dt.strftime('%Y-%m')
df_vendas['valor_imposto'] = pd.to_numeric(df_vendas['valor_imposto'])/100
df_vendas['qtd_vendida'] = pd.to_numeric(df_vendas['qtd_vendida'])


Unir Produtos com Vendas

In [14]:
df_vendas_item = pd.merge(df_vendas,df_produtos, on='cod_produto')
df_vendas_item['Total'] = df_vendas_item['qtd_vendida'] * df_vendas_item['valor_unitario']
df_vendas_item['Total_peso'] = df_vendas_item['qtd_vendida'] * df_vendas_item['peso']
df_vendas_item['Total_imposto'] = df_vendas_item['Total'] * df_vendas_item['valor_imposto']
df_vendas_item['Total_liquido'] = df_vendas_item['Total'] - df_vendas_item['Total_imposto']
df_vendas_item['ano_mes']  = df_vendas_item['data_venda'].dt.strftime('%Y/%m')

Unir Produtos_vendas com custos e taxas

In [15]:
df_geral = pd.merge(df_vendas_item,df_merge,on=['cod_produto','ano_mes'])

Calcula valores

In [28]:
df_geral['un_liquido'] = df_geral['Total_liquido']/df_geral['qtd_vendida']
df_geral['total_mdo'] = df_geral['mao_de_obra']*df_geral['qtd_vendida']
df_geral['total_mp'] = df_geral['materia_prima']*df_geral['qtd_vendida']
df_geral['un_mdi'] = df_geral['mao_de_obra_indireta']*df_geral['gasto_geral']
df_geral['un_energia'] = df_geral['energia']*df_geral['gasto_geral']
df_geral['un_manutencao'] = df_geral['manutencao']*df_geral['gasto_geral']
df_geral['un_depreciacao'] = df_geral['depreciacao']*df_geral['gasto_geral']
df_geral['total_mdi'] = df_geral['un_mdi']*df_geral['qtd_vendida']
df_geral['total_energia'] = df_geral['un_energia']*df_geral['qtd_vendida']
df_geral['total_manutencao'] = df_geral['un_manutencao']*df_geral['qtd_vendida']
df_geral['total_depreciacao'] = df_geral['un_depreciacao']*df_geral['qtd_vendida']
df_geral['un_lucro_bruto'] = df_geral['un_liquido']-df_geral['mao_de_obra']-df_geral['materia_prima']-df_geral['energia']
df_geral['lucro_bruto'] = df_geral['un_lucro_bruto']*df_geral['qtd_vendida']
df_geral['un_r_financeiro'] =  df_geral['lucro_bruto']-df_geral['un_manutencao']-df_geral['un_mdi']-df_geral['un_depreciacao']
df_geral['r_financeiro'] = df_geral['un_r_financeiro']*df_geral['qtd_vendida']

Adiciona no banco de dados

In [45]:
connection = pymysql.connect(host='localhost', port=3306, user='root', password='', db='xpeducacao')
table_name = 'projetofinal'
df_geral.to_sql(table_name, connection, if_exists='replace')

C:\Users\Anderson\AppData\Local\Temp\ipykernel_15104\2495982108.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_geral.to_sql(table_name, connection, if_exists='replace')


DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': not all arguments converted during string formatting

In [ ]:
df_geral.head(10)